In [20]:
import arcpy
import os
import math
import datetime
import uuid

In [21]:
def find_field_by_pattern(fields, pattern_list):
    """Helper function to find a field based on patterns"""
    for field in fields:
        field_name = field.name.upper()
        if all(pattern in field_name for pattern in pattern_list):
            return field.name
    return None

def find_exact_field(fields, name):
    """Helper function to find a field based on exact name"""
    for field in fields:
        if field.name.upper() == name.upper():
            return field.name
    return None

In [22]:
# Ask user for threshold values
print("====== PCI Difference Threshold Input ======")
lower_threshold = float(input("Enter LOWER threshold value (include points with PCI difference <= this value): "))
higher_threshold = float(input("Enter HIGHER threshold value (include points with PCI difference >= this value): "))
print(f"Will include points with PCI difference <= {lower_threshold} OR >= {higher_threshold}")

# New input for work history filtering
work_history_filter = input("Do you want to only include rows above the positive threshold that have NO work history? (Yes/No): ")
work_history_filter = work_history_filter.lower().strip()
if work_history_filter == 'yes':
    print("Will only include rows above the positive threshold that have NO work history (empty MRTreatmentName)")
else:
    print("Will include all rows that meet threshold criteria regardless of work history")
print("============================================")

====== PCI Difference Threshold Input ======
Enter LOWER threshold value (include points with PCI difference <= this value): -10
Enter HIGHER threshold value (include points with PCI difference >= this value): 15
Will include points with PCI difference <= -10.0 OR >= 15.0
Do you want to only include rows above the positive threshold that have NO work history? (Yes/No): No
Will include all rows that meet threshold criteria regardless of work history


In [23]:
# Get the current project
aprx = arcpy.mp.ArcGISProject("CURRENT")
print(f"Current project: {aprx.filePath}")

# Get the active map and find the WildomarPMPJordan feature layer
PMP_Path = None
active_map = None

# Search for the feature class in all maps
for m in aprx.listMaps():
    for lyr in m.listLayers():
        if lyr.isFeatureLayer and "SSstreet" in lyr.name:
            PMP_Path = lyr.dataSource
            wildomar_layer = lyr
            active_map = m
            print(f"Found layer: {lyr.name} in map: {m.name}")
            break
    if PMP_Path:
        break

if not PMP_Path:
    print("ERROR: WildomarPMPJordan feature class not found in the project.")
    # raise RuntimeError("WildomarPMPJordan feature class not found in the project.")

# Define WGS84 (lat/long) spatial reference for external API URLs
wgs84 = arcpy.SpatialReference(4326)  # EPSG code for WGS84

# Get the spatial reference of the feature layer
desc = arcpy.Describe(wildomar_layer)
layer_sr = desc.spatialReference

print(f"Feature layer spatial reference: {layer_sr.name} ({layer_sr.factoryCode})")
print(f"Target spatial reference: WGS84 (4326)")


Current project: C:\Users\JLin\Downloads\QC Tool\QC Tool.aprx
Found layer: SSstreet in map: Map
Feature layer spatial reference: NAD_1983_StatePlane_California_VI_FIPS_0406_Feet (2230)
Target spatial reference: WGS84 (4326)


In [24]:
# Find the PCI Differences table
pci_table = None
for table in active_map.listTables():
    if "PCI Differences" in table.name:
        pci_table = table
        print(f"Found standalone table: {table.name}")
        break

if not pci_table:
    print("WARNING: PCI Differences table not found in the active map.")
    # Use a raise statement here if you want to stop execution

Found standalone table: PCI Differences


In [25]:
# Determine field names in the feature layer
print("\nLooking for key fields in the feature layer...")
shapefile_fields = arcpy.ListFields(wildomar_layer)

# Use exact field names from your data
street_id_field = find_exact_field(shapefile_fields, "Street_ID")
section_id_field = find_exact_field(shapefile_fields, "Section_ID")
street_name_field = find_exact_field(shapefile_fields, "Street_Name")
begin_loc_field = find_exact_field(shapefile_fields, "Begin_Location")
end_loc_field = find_exact_field(shapefile_fields, "End_Location")

# Print found fields with clear confirmation
if street_id_field:
    print(f"Found street ID field: {street_id_field}")
else:
    print("WARNING: Street_ID field not found in feature layer")
    
if section_id_field:
    print(f"Found section ID field: {section_id_field}")
else:
    print("WARNING: Section_ID field not found in feature layer")
    
if street_name_field:
    print(f"Found street name field: {street_name_field}")
else:
    print("WARNING: Street_Name field not found in feature layer")
    
if begin_loc_field:
    print(f"Found begin location field: {begin_loc_field}")
else:
    print("WARNING: Begin_Location field not found in feature layer")
    
if end_loc_field:
    print(f"Found end location field: {end_loc_field}")
else:
    print("WARNING: End_Location field not found in feature layer")


Looking for key fields in the feature layer...


In [26]:
# Determine field names in the PCI table
print("\nLooking for key fields in the PCI Differences table...")
pci_fields = arcpy.ListFields(pci_table)

# Use exact field names from your screenshot
pci_street_id_field = find_exact_field(pci_fields, "Street_ID")
pci_section_id_field = find_exact_field(pci_fields, "Section_ID")
prev_insp_pci_field = find_exact_field(pci_fields, "Prev_Insp_PCI")
last_insp_pci_field = find_exact_field(pci_fields, "Last_Insp_PCI")
prev_insp_date_field = find_exact_field(pci_fields, "Prev_Insp_Date")
m_r_date_field = find_exact_field(pci_fields, "M_R_Date")
m_r_treatment_field = find_exact_field(pci_fields, "M_R_Treatment_Name")
m_r_pci_field = find_exact_field(pci_fields, "M_R_PCI")
last_insp_date_field = find_exact_field(pci_fields, "Last_Insp_Date")
begin_loc_field_pci = find_exact_field(pci_fields, "Begin_Location")
end_loc_field_pci = find_exact_field(pci_fields, "End_Location")

# Print status of each field
for field_name, field_var in [
    ("Street_ID", pci_street_id_field),
    ("Section_ID", pci_section_id_field),
    ("Prev_Insp_PCI", prev_insp_pci_field),
    ("Last_Insp_PCI", last_insp_pci_field),
    ("Prev_Insp_Date", prev_insp_date_field),
    ("M_R_Date", m_r_date_field),
    ("M_R_Treatment_Name", m_r_treatment_field),
    ("M_R_PCI", m_r_pci_field),
    ("Last_Insp_Date", last_insp_date_field),
    ("Begin_Location", begin_loc_field_pci),
    ("End_Location", end_loc_field_pci)
]:
    if field_var:
        print(f"Found field: {field_name} = {field_var}")
    else:
        print(f"WARNING: {field_name} field not found in PCI table")

# Check if we have the necessary fields
if not pci_street_id_field or not pci_section_id_field:
    print("ERROR: Could not find Street ID and Section ID fields in the PCI table.")
    # Use a raise statement here if you want to stop execution

if not street_id_field and not section_id_field:
    print("ERROR: Could not find Street ID or Section ID fields in the feature layer.")
    # Use a raise statement here if you want to stop execution

# Check if PCI fields were found in the table
if not prev_insp_pci_field or not last_insp_pci_field:
    print("ERROR: Could not find Prev_Insp_PCI or Last_Insp_PCI fields in the PCI table.")
    # Use a raise statement here if you want to stop execution
else:
    print(f"Found both PCI fields in the table. Will calculate difference: {prev_insp_pci_field} - {last_insp_pci_field}")


Looking for key fields in the PCI Differences table...
Found field: Street_ID = Street_ID
Found field: Section_ID = Section_ID
Found field: Prev_Insp_PCI = Prev_Insp_PCI
Found field: Last_Insp_PCI = Last_Insp_PCI
Found field: Prev_Insp_Date = Prev_Insp_Date
Found field: M_R_Date = M_R_Date
Found field: M_R_Treatment_Name = M_R_Treatment_Name
Found field: M_R_PCI = M_R_PCI
Found field: Last_Insp_Date = Last_Insp_Date
Found field: Begin_Location = Begin_Location
Found field: End_Location = End_Location
ERROR: Could not find Street ID or Section ID fields in the feature layer.
Found both PCI fields in the table. Will calculate difference: Prev_Insp_PCI - Last_Insp_PCI


In [27]:
# Create a dictionary to store PCI data for quicker lookups
print("\nReading PCI Differences data...")

# Debug: check a few records first
print("\nExamining PCI data types and values...")
sample_count = 0
sample_fields = [pci_street_id_field, pci_section_id_field]
if prev_insp_pci_field:
    sample_fields.append(prev_insp_pci_field)
if last_insp_pci_field:
    sample_fields.append(last_insp_pci_field)

with arcpy.da.SearchCursor(pci_table, sample_fields) as cursor:
    for row in cursor:
        street_id = str(row[0]).strip() if row[0] else "None"
        section_id = str(row[1]).strip() if row[1] else "None"
        
        print(f"Sample {sample_count}: {street_id}-{section_id}")
        field_idx = 2
        if prev_insp_pci_field:
            prev_pci = row[field_idx]
            print(f"  Prev PCI: {prev_pci} (type: {type(prev_pci)})")
            field_idx += 1
        if last_insp_pci_field:
            last_pci = row[field_idx]
            print(f"  Last PCI: {last_pci} (type: {type(last_pci)})")
        
        sample_count += 1
        if sample_count >= 5:  # Just show a few samples
            break

pci_data = {}

# Determine which fields to fetch from the PCI table
pci_fields_to_fetch = [pci_street_id_field, pci_section_id_field]

# Add required fields if they exist
if prev_insp_pci_field:
    pci_fields_to_fetch.append(prev_insp_pci_field)
if last_insp_pci_field:
    pci_fields_to_fetch.append(last_insp_pci_field)
if prev_insp_date_field:
    pci_fields_to_fetch.append(prev_insp_date_field)
if m_r_date_field:
    pci_fields_to_fetch.append(m_r_date_field)
if m_r_treatment_field:
    pci_fields_to_fetch.append(m_r_treatment_field)
if m_r_pci_field:
    pci_fields_to_fetch.append(m_r_pci_field)
if last_insp_date_field:
    pci_fields_to_fetch.append(last_insp_date_field)
# Add begin/end location fields from PCI table if they exist
if begin_loc_field_pci:
    pci_fields_to_fetch.append(begin_loc_field_pci)
if end_loc_field_pci:
    pci_fields_to_fetch.append(end_loc_field_pci)

with arcpy.da.SearchCursor(pci_table, pci_fields_to_fetch) as cursor:
    for row in cursor:
        street_id = str(row[0]).strip() if row[0] else ""
        section_id = str(row[1]).strip() if row[1] else ""
        
        # Create key in "Street_ID - Section_ID" format
        key = f"{street_id} - {section_id}"
        
        record_data = {
            'street_id': street_id,
            'section_id': section_id
        }
        
        # Get field index
        field_idx = 2
        
        # Get PCI values if fields are available
        if prev_insp_pci_field:
            record_data['prev_pci'] = row[field_idx]
            field_idx += 1
        if last_insp_pci_field:
            record_data['last_pci'] = row[field_idx]
            field_idx += 1
        
        # Get date and treatment fields
        if prev_insp_date_field:
            record_data['prev_insp_date'] = row[field_idx]
            field_idx += 1
        if m_r_date_field:
            record_data['m_r_date'] = row[field_idx]
            field_idx += 1
        if m_r_treatment_field:
            record_data['m_r_treatment'] = row[field_idx]
            field_idx += 1
        if m_r_pci_field:
            record_data['m_r_pci'] = row[field_idx]
            field_idx += 1
        if last_insp_date_field:
            record_data['last_insp_date'] = row[field_idx]
            field_idx += 1
        
        # Get begin/end location fields from PCI table if they exist
        if begin_loc_field_pci:
            record_data['begin_loc_pci'] = row[field_idx]
            field_idx += 1
        if end_loc_field_pci:
            record_data['end_loc_pci'] = row[field_idx]
            field_idx += 1
        
        # Calculate PCI difference if both PCI fields are available
        if 'prev_pci' in record_data and 'last_pci' in record_data:
            prev_pci = record_data['prev_pci']
            last_pci = record_data['last_pci']
            
            # Check for valid numeric values before calculating
            if prev_pci is not None and last_pci is not None:
                try:
                    # Try both calculation methods
                    pci_diff_calc = float(prev_pci) - float(last_pci)
                    record_data['pci_diff_calc'] = pci_diff_calc
                    
                    # Check if the record meets threshold criteria
                    meets_threshold = False
                    
                    # For lower threshold, we include regardless of work history
                    if pci_diff_calc <= lower_threshold:
                        meets_threshold = True
                        print(f"Record {key} included: PCI difference {pci_diff_calc} <= {lower_threshold}")
                    
                    # For higher threshold, we may filter based on work history
                    elif pci_diff_calc >= higher_threshold:
                        if work_history_filter.lower().strip() == 'yes':
                            # Check if there is NO work history (empty treatment field)
                            has_work_history = 'm_r_treatment' in record_data and record_data['m_r_treatment']
                            if not has_work_history:
                                meets_threshold = True
                                print(f"Record {key} included: PCI difference {pci_diff_calc} >= {higher_threshold} with NO work history")
                            else:
                                print(f"Record {key} excluded: PCI difference {pci_diff_calc} >= {higher_threshold} but HAS work history")
                        else:
                            # Include all records above higher threshold
                            meets_threshold = True
                            print(f"Record {key} included: PCI difference {pci_diff_calc} >= {higher_threshold}")
                    else:
                        print(f"Record {key} excluded: PCI difference {pci_diff_calc} not within thresholds")
                    
                    # Store record if it meets criteria
                    if meets_threshold:
                        record_data['meets_criteria'] = True
                        pci_data[key] = record_data
                    else:
                        record_data['meets_criteria'] = False
                        pci_data[key] = record_data
                        
                except (ValueError, TypeError):
                    print(f"Warning: Could not calculate PCI difference for {key}. Values: {prev_pci}, {last_pci}")

print(f"Loaded {len([k for k, v in pci_data.items() if v.get('meets_criteria', True)])} records from PCI Differences table that meet threshold criteria.")


Reading PCI Differences data...

Examining PCI data types and values...
Sample 0: 100-1076
  Prev PCI: 74 (type: <class 'int'>)
  Last PCI: 60 (type: <class 'int'>)
Sample 1: 100-3084
  Prev PCI: 72 (type: <class 'int'>)
  Last PCI: 73 (type: <class 'int'>)
Sample 2: 100-3229
  Prev PCI: 77 (type: <class 'int'>)
  Last PCI: 69 (type: <class 'int'>)
Sample 3: 100-3232
  Prev PCI: 60 (type: <class 'int'>)
  Last PCI: 75 (type: <class 'int'>)
Sample 4: 100-3234
  Prev PCI: 78 (type: <class 'int'>)
  Last PCI: 59 (type: <class 'int'>)
Record 100 - 1076 excluded: PCI difference 14.0 not within thresholds
Record 100 - 3084 excluded: PCI difference -1.0 not within thresholds
Record 100 - 3229 excluded: PCI difference 8.0 not within thresholds
Record 100 - 3232 included: PCI difference -15.0 <= -10.0
Record 100 - 3234 included: PCI difference 19.0 >= 15.0
Record 100 - 7032 included: PCI difference 17.0 >= 15.0
Record 100 - 7448 excluded: PCI difference -6.0 not within thresholds
Record 1000 -

Record 1079 - 5620 excluded: PCI difference -4.0 not within thresholds
Record 1080 - 1799 excluded: PCI difference -5.0 not within thresholds
Record 1082 - 4046 excluded: PCI difference -4.0 not within thresholds
Record 1084 - 2298 excluded: PCI difference -1.0 not within thresholds
Record 1086 - 2301 excluded: PCI difference -1.0 not within thresholds
Record 1088 - 1340 included: PCI difference -24.0 <= -10.0
Record 1090 - 2048 included: PCI difference -15.0 <= -10.0
Record 1092 - 4794 excluded: PCI difference 0.0 not within thresholds
Record 1092 - 7124 excluded: PCI difference -1.0 not within thresholds
Record 1094 - 1631 excluded: PCI difference -1.0 not within thresholds
Record 1094 - 2606 excluded: PCI difference -4.0 not within thresholds
Record 1096 - 3412 excluded: PCI difference -3.0 not within thresholds
Record 1096 - 5056 excluded: PCI difference -4.0 not within thresholds
Record 1096 - 5057 excluded: PCI difference -4.0 not within thresholds
Record 1096 - 5058 included: PC

Record 1170 - 4279 excluded: PCI difference 13.0 not within thresholds
Record 1170 - 4280 included: PCI difference -11.0 <= -10.0
Record 1170 - 4281 excluded: PCI difference -3.0 not within thresholds
Record 1172 - 4514 included: PCI difference -71.0 <= -10.0
Record 1174 - 7482 included: PCI difference -23.0 <= -10.0
Record 1176 - 3497 included: PCI difference -28.0 <= -10.0
Record 1176 - 3503 included: PCI difference -21.0 <= -10.0
Record 1177 - 6499 excluded: PCI difference -3.0 not within thresholds
Record 1177 - 7526 excluded: PCI difference -2.0 not within thresholds
Record 1178 - 3351 included: PCI difference -49.0 <= -10.0
Record 1180 - 3354 included: PCI difference -65.0 <= -10.0
Record 1180 - 3355 included: PCI difference -49.0 <= -10.0
Record 1182 - 1178 excluded: PCI difference -4.0 not within thresholds
Record 1184 - 1590 excluded: PCI difference -4.0 not within thresholds
Record 1186 - 4456 excluded: PCI difference -4.0 not within thresholds
Record 1188 - 1249 excluded: PC

Record 1284 - 5574 excluded: PCI difference -5.0 not within thresholds
Record 1286 - 5575 excluded: PCI difference -4.0 not within thresholds
Record 1288 - 3961 excluded: PCI difference -3.0 not within thresholds
Record 1290 - 2958 excluded: PCI difference -2.0 not within thresholds
Record 1290 - 7077 excluded: PCI difference -2.0 not within thresholds
Record 1292 - 3462 excluded: PCI difference -4.0 not within thresholds
Record 1292 - 4205 excluded: PCI difference -4.0 not within thresholds
Record 1292 - 7076 excluded: PCI difference -2.0 not within thresholds
Record 1294 - 2546 included: PCI difference -16.0 <= -10.0
Record 1296 - 6528 excluded: PCI difference 0.0 not within thresholds
Record 1298 - 6595 included: PCI difference 18.0 >= 15.0
Record 1298 - 6596 excluded: PCI difference 11.0 not within thresholds
Record 1298 - 6631 excluded: PCI difference 8.0 not within thresholds
Record 1298 - 6809 excluded: PCI difference 13.0 not within thresholds
Record 130 - 10798 excluded: PCI d

Record 135 - 5319 excluded: PCI difference -2.0 not within thresholds
Record 135 - 5320 excluded: PCI difference -8.0 not within thresholds
Record 135 - 5345 included: PCI difference -11.0 <= -10.0
Record 135 - 5989 excluded: PCI difference -4.0 not within thresholds
Record 1350 - 5852 excluded: PCI difference -9.0 not within thresholds
Record 1352 - 7218 excluded: PCI difference -3.0 not within thresholds
Record 1354 - 2592 excluded: PCI difference -4.0 not within thresholds
Record 1356 - 1865 excluded: PCI difference -7.0 not within thresholds
Record 1358 - 1754 excluded: PCI difference -5.0 not within thresholds
Record 1358 - 6835 excluded: PCI difference -6.0 not within thresholds
Record 1360 - 1773 excluded: PCI difference -9.0 not within thresholds
Record 1362 - 1638 excluded: PCI difference -3.0 not within thresholds
Record 1362 - 6264 excluded: PCI difference -3.0 not within thresholds
Record 1362 - 6265 excluded: PCI difference -3.0 not within thresholds
Record 1364 - 1515 inc

...

Record 1794 - 1680 included: PCI difference -13.0 <= -10.0
Record 1794 - 4867 excluded: PCI difference -5.0 not within thresholds
Record 1794 - 5650 excluded: PCI difference -4.0 not within thresholds
Record 1794 - 5651 excluded: PCI difference -5.0 not within thresholds
Record 1796 - 1629 excluded: PCI difference -3.0 not within thresholds
Record 1798 - 2038 excluded: PCI difference -5.0 not within thresholds
Record 180 - 7498 excluded: PCI difference 9.0 not within thresholds
Record 1800 - 4639 excluded: PCI difference 14.0 not within thresholds
Record 1800 - 4640 excluded: PCI difference -7.0 not within thresholds
Record 1802 - 5481 excluded: PCI difference 0.0 not within thresholds
Record 1804 - 6778 excluded: PCI difference -5.0 not within thresholds
Record 1804 - 7129 excluded: PCI difference -3.0 not within thresholds
Record 1806 - 1372 excluded: PCI difference 9.0 not within thresholds
Record 1808 - 2376 included: PCI difference 31.0 >= 15.0
Record 1810 - 5436 excluded: PCI dif

Record 2436 - 6625 excluded: PCI difference -4.0 not within thresholds
Record 2438 - 2237 excluded: PCI difference -4.0 not within thresholds
Record 2440 - 6304 included: PCI difference -14.0 <= -10.0
Record 2442 - 1387 included: PCI difference -62.0 <= -10.0
Record 2444 - 5974 excluded: PCI difference 3.0 not within thresholds
Record 2444 - 6039 excluded: PCI difference 12.0 not within thresholds
Record 2446 - 4556 excluded: PCI difference 10.0 not within thresholds
Record 2446 - 4557 included: PCI difference 16.0 >= 15.0
Record 2446 - 4558 excluded: PCI difference 6.0 not within thresholds
Record 2448 - 4841 excluded: PCI difference -2.0 not within thresholds
Record 245 - 1071 excluded: PCI difference 8.0 not within thresholds
Record 2450 - 3549 excluded: PCI difference -3.0 not within thresholds
Record 2452 - 2134 included: PCI difference -14.0 <= -10.0
Record 2454 - 1309 excluded: PCI difference -5.0 not within thresholds
Record 2454 - 3964 excluded: PCI difference -6.0 not within 

Record 2518 - 5408 included: PCI difference 41.0 >= 15.0
Record 2520 - 6139 included: PCI difference -11.0 <= -10.0
Record 2522 - 3629 included: PCI difference -39.0 <= -10.0
Record 2522 - 5249 excluded: PCI difference 7.0 not within thresholds
Record 2524 - 7178 excluded: PCI difference -6.0 not within thresholds
Record 2526 - 2209 included: PCI difference -14.0 <= -10.0
Record 2528 - 3709 excluded: PCI difference 14.0 not within thresholds
Record 2530 - 3960 excluded: PCI difference -6.0 not within thresholds
Record 2532 - 5772 excluded: PCI difference -4.0 not within thresholds
Record 2534 - 7201 excluded: PCI difference 8.0 not within thresholds
Record 2536 - 4678 excluded: PCI difference 6.0 not within thresholds
Record 2538 - 4677 included: PCI difference 36.0 >= 15.0
Record 2538 - 5514 excluded: PCI difference 4.0 not within thresholds
Record 2538 - 5515 excluded: PCI difference 0.0 not within thresholds
Record 2540 - 6081 excluded: PCI difference -1.0 not within thresholds
Reco

Record 265 - 1007 excluded: PCI difference 7.0 not within thresholds
Record 265 - 2060 excluded: PCI difference -4.0 not within thresholds
Record 265 - 5764 excluded: PCI difference -8.0 not within thresholds
Record 265 - 6114 excluded: PCI difference -3.0 not within thresholds
Record 265 - 6115 excluded: PCI difference -1.0 not within thresholds
Record 265 - 6116 excluded: PCI difference 1.0 not within thresholds
Record 265 - 7510 excluded: PCI difference 2.0 not within thresholds
Record 2650 - 1711 excluded: PCI difference -4.0 not within thresholds
Record 2652 - 2184 excluded: PCI difference -2.0 not within thresholds
Record 2654 - 3855 excluded: PCI difference 3.0 not within thresholds
Record 2654 - 3856 excluded: PCI difference 12.0 not within thresholds
Record 2654 - 3858 included: PCI difference 21.0 >= 15.0
Record 2654 - 5935 excluded: PCI difference 14.0 not within thresholds
Record 2654 - 5936 included: PCI difference 17.0 >= 15.0
Record 2656 - 1548 excluded: PCI difference -

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
# Outputs in default GDB
default_gdb = aprx.defaultGeodatabase
print(f"\nCreating outputs in default geodatabase: {default_gdb}")

# Create more user-friendly names
current_date = datetime.datetime.now().strftime("%Y%m%d")
formatted_thresholds = f"PCI_Diff_LE_{lower_threshold}_GE_{higher_threshold}"

# More recognizable names for outputs
table_name = f"QC_Table"
fc_name = f"QC_Points"
joined_fc_name = f"QC_Joined_Shapefile"

table_path = os.path.join(default_gdb, table_name)
fc_path = os.path.join(default_gdb, fc_name)
joined_fc_path = os.path.join(default_gdb, joined_fc_name)

# Thorough cleanup of ALL existing outputs with similar names to avoid duplicates
print("\nPerforming thorough cleanup of existing layers and tables...")

# Start by removing any matching layers from ALL maps in the project
for m in aprx.listMaps():
    print(f"Checking map: {m.name}")
    
    # Get lists of layers and tables to remove (collect first, remove after)
    layers_to_remove = []
    tables_to_remove = []
    
    # Check all layers
    for lyr in m.listLayers():
        try:
            if any(pattern in lyr.name for pattern in ["PCI_Threshold_Points", "PCI_Joined", "Midpoints", "TEMP_Midpoints"]):
                layers_to_remove.append(lyr)
                print(f"Will remove layer: {lyr.name}")
        except AttributeError:
            # Handle layers without valid name attribute
            layers_to_remove.append(lyr)
            print("Found a layer with no valid name attribute - will try to remove")
    
    # Check all tables
    for tbl in m.listTables():
        try:
            if "PCI_Threshold_Table" in tbl.name:
                tables_to_remove.append(tbl)
                print(f"Will remove table: {tbl.name}")
        except AttributeError:
            # Handle tables without valid name attribute
            tables_to_remove.append(tbl)
            print("Found a table with no valid name attribute - will try to remove")
    
    # Remove all identified layers
    for lyr in layers_to_remove:
        try:
            m.removeLayer(lyr)
            print(f"Removed layer from map")
        except Exception as e:
            print(f"Warning: Could not remove layer. Error: {str(e)}")
    
    # Remove all identified tables
    for tbl in tables_to_remove:
        try:
            m.removeTable(tbl)
            print(f"Removed table from map")
        except Exception as e:
            print(f"Warning: Could not remove table. Error: {str(e)}")

# Delete ALL matching feature classes and tables in the geodatabase
arcpy_workspace = aprx.defaultGeodatabase
arcpy.env.workspace = arcpy_workspace

# Define patterns to match ALL output types
pattern_list = [
    "*PCI_Threshold_Points*", 
    "*PCI_Threshold_Table*", 
    "*Midpoints*", 
    "*TEMP_Midpoints*",
    "*PCI_Joined*"
]

# Delete all items matching any pattern
for pattern in pattern_list:
    # Try to delete as feature classes
    for item in arcpy.ListFeatureClasses(pattern):
        try:
            item_path = os.path.join(arcpy_workspace, item)
            arcpy.management.Delete(item_path)
            print(f"Deleted existing feature class: {item}")
        except Exception as e:
            print(f"Warning: Could not delete feature class {item}. Error: {str(e)}")
    
    # Try to delete as tables
    for item in arcpy.ListTables(pattern):
        try:
            item_path = os.path.join(arcpy_workspace, item)
            arcpy.management.Delete(item_path)
            print(f"Deleted existing table: {item}")
        except Exception as e:
            print(f"Warning: Could not delete table {item}. Error: {str(e)}")

print("Cleanup complete - all previous outputs removed")

In [29]:
# Create a unique name for the temporary midpoints
unique_id = str(uuid.uuid4())[:8]
midpoints_fc_name = f"TEMP_Midpoints_{unique_id}"
midpoints_fc_path = os.path.join(default_gdb, midpoints_fc_name)

# Create midpoints using the FeatureToPoint tool
print("\nGenerating midpoints using FeatureToPoint tool...")
if arcpy.Exists(midpoints_fc_path):
    arcpy.management.Delete(midpoints_fc_path)
    
arcpy.management.FeatureToPoint(
    PMP_Path, 
    midpoints_fc_path, 
    "CENTROID"  # Using CENTROID for lines will give the midpoint
)

print(f"Midpoints created successfully at: {midpoints_fc_path}")


Generating midpoints using FeatureToPoint tool...
Midpoints created successfully at: C:\Users\JLin\Downloads\QC Tool\Default.gdb\TEMP_Midpoints_86bf9fd9


In [30]:
# ================================================================
# Cell 11 – Extract attributes from original feature class
# ================================================================

print("\nExtracting attributes from original feature class...")

# Check if required variables are defined
if 'street_sec_field' not in locals() or street_sec_field is None:
    print("Warning: street_sec_field not defined. Will use separate ID fields.")
    use_combined_field = False
else:
    use_combined_field = True
    print(f"Using combined field: {street_sec_field}")

if 'street_id_field' not in locals() or street_id_field is None:
    print("Error: street_id_field not defined!")
    street_id_field = find_exact_field(arcpy.ListFields(wildomar_layer), "StreetID")
    print(f"Attempting to auto-detect: {street_id_field}")

if 'section_id_field' not in locals() or section_id_field is None:
    print("Error: section_id_field not defined!")
    section_id_field = find_exact_field(arcpy.ListFields(wildomar_layer), "SectionID")
    print(f"Attempting to auto-detect: {section_id_field}")

if 'street_name_field' not in locals() or street_name_field is None:
    print("Warning: street_name_field not defined.")
    street_name_field = find_exact_field(arcpy.ListFields(wildomar_layer), "STREETNAME")
    print(f"Attempting to auto-detect: {street_name_field}")

if 'begin_loc_field' not in locals() or begin_loc_field is None:
    print("Warning: begin_loc_field not defined.")
    begin_loc_field = find_exact_field(arcpy.ListFields(wildomar_layer), "BegLocatio")
    print(f"Attempting to auto-detect: {begin_loc_field}")

if 'end_loc_field' not in locals() or end_loc_field is None:
    print("Warning: end_loc_field not defined.")
    end_loc_field = find_exact_field(arcpy.ListFields(wildomar_layer), "EndLocatio")
    print(f"Attempting to auto-detect: {end_loc_field}")

# Determine fields to retrieve from the original feature class
shapefile_retrieve_fields = ["SHAPE@"]

# If we have a combined field, use it
if use_combined_field:
    shapefile_retrieve_fields.append(street_sec_field)
# Otherwise use separate fields
else:
    if street_id_field:
        shapefile_retrieve_fields.append(street_id_field)
    if section_id_field:
        shapefile_retrieve_fields.append(section_id_field)
    
# Add additional fields if they exist
if street_name_field:
    shapefile_retrieve_fields.append(street_name_field)
if begin_loc_field:
    shapefile_retrieve_fields.append(begin_loc_field)
if end_loc_field:
    shapefile_retrieve_fields.append(end_loc_field)

print(f"Fields to retrieve: {shapefile_retrieve_fields}")

# Join fields from original data with midpoint geometries
# First, create a dictionary of attributes from the original feature class
attributes_dict = {}

try:
    with arcpy.da.SearchCursor(wildomar_layer, shapefile_retrieve_fields) as cursor:
        for i, row in enumerate(cursor):
            # Get the shape
            shape = row[0]  # SHAPE@
            
            # Initialize with defaults
            street_id_value = ""
            section_id_value = ""
            street_name_value = ""
            begin_loc_value = ""
            end_loc_value = ""
            
            field_idx = 1  # Start from field after SHAPE@
            
            # Determine the key based on available fields
            if use_combined_field:
                street_sec_value = str(row[field_idx]).strip() if row[field_idx] else ""
                key = street_sec_value
                street_id_value = street_sec_value
                section_id_value = street_sec_value
                field_idx += 1
            else:
                # Safely get values with error handling
                if street_id_field and field_idx < len(row):
                    street_id_value = str(row[field_idx]).strip() if row[field_idx] else ""
                    field_idx += 1
                if section_id_field and field_idx < len(row):
                    section_id_value = str(row[field_idx]).strip() if row[field_idx] else ""
                    field_idx += 1
                
                key = f"{street_id_value} - {section_id_value}"
            
            # Get additional fields if they exist
            if street_name_field and field_idx < len(row):
                street_name_value = row[field_idx] if row[field_idx] else ""
                field_idx += 1
            if begin_loc_field and field_idx < len(row):
                begin_loc_value = row[field_idx] if row[field_idx] else ""
                field_idx += 1
            if end_loc_field and field_idx < len(row):
                end_loc_value = row[field_idx] if row[field_idx] else ""
                field_idx += 1
            
            # Calculate midpoint
            midpoint = shape.positionAlongLine(0.5, True)
            
            # Project the point to WGS84 for API URLs
            proj = midpoint.projectAs(wgs84)
            lon = proj.firstPoint.X
            lat = proj.firstPoint.Y
            
            # Store original coordinates
            original_x = midpoint.firstPoint.X
            original_y = midpoint.firstPoint.Y
            
            # Store attributes
            attributes_dict[i] = {
                'street_id': street_id_value,
                'section_id': section_id_value,
                'combined_key': key,
                'street_name': street_name_value,
                'begin_loc': begin_loc_value,
                'end_loc': end_loc_value,
                'latitude': lat,
                'longitude': lon,
                'original_x': original_x,
                'original_y': original_y,
                'spatial_reference': layer_sr
            }
            
            # Print progress every 500 records
            if i % 500 == 0:
                print(f"Processed {i} records...")
    
    print(f"Successfully extracted attributes for {len(attributes_dict)} features")
    
    # Sample a few keys for debugging
    print("Sample keys from attributes_dict:")
    sample_count = 0
    for i, attrs in attributes_dict.items():
        if sample_count < 5:
            print(f"Key {attrs['combined_key']}: Street ID={attrs['street_id']}, Section ID={attrs['section_id']}")
            sample_count += 1

except Exception as e:
    print(f"Error extracting attributes: {str(e)}")
    import traceback
    traceback.print_exc()


Extracting attributes from original feature class...
Error: street_id_field not defined!
Attempting to auto-detect: StreetID
Error: section_id_field not defined!
Attempting to auto-detect: SectionID
Attempting to auto-detect: STREETNAME
Attempting to auto-detect: BegLocatio
Attempting to auto-detect: EndLocatio
Fields to retrieve: ['SHAPE@', 'StreetID', 'SectionID', 'STREETNAME', 'BegLocatio', 'EndLocatio']
Processed 0 records...
Processed 500 records...
Processed 1000 records...
Processed 1500 records...
Processed 2000 records...
Processed 2500 records...
Processed 3000 records...
Processed 3500 records...
Processed 4000 records...
Processed 4500 records...
Processed 5000 records...
Processed 5500 records...
Processed 6000 records...
Successfully extracted attributes for 6334 features
Sample keys from attributes_dict:
Key 1002 - 5466: Street ID=1002, Section ID=5466
Key 1002 - 5467: Street ID=1002, Section ID=5467
Key 1004 - 4057: Street ID=1004, Section ID=4057
Key 1004 - 5279: Stre

In [31]:
# ================================================================
# Cell 11 – Build output records with QC_Points field names
# ================================================================

print("\n🔎 Matching attributes with PCI data (all features)…")

results = []

def safe_float(v):
    try:
        return float(v)
    except (TypeError, ValueError):
        return None

for fid, attrs in attributes_dict.items():
    key = attrs["combined_key"]
    rec = {}                               # fresh dict per line feature
    rec.update(attrs)                      # copy original line attributes

    # ---- Pull PCI‑history values -----------------------------------------
    pci = pci_data.get(key)                # may be None
    
    # Debug output for key records
    if pci and pci.get('meets_criteria', False):
        print(f"DEBUG: Found matching criteria record for {key}")
        
    prev_date   = pci.get('prev_insp_date')      if pci else None
    prev_pci    = safe_float(pci.get('prev_pci')) if pci else None
    mr_date     = pci.get('m_r_date')           if pci else None
    mr_treat    = pci.get('m_r_treatment')      if pci else None
    last_date   = pci.get('last_insp_date')     if pci else None
    last_pci    = safe_float(pci.get('last_pci')) if pci else None
    pci_diff    = safe_float(pci.get('pci_diff_calc')) if pci else None

    rec["PrevInspDate"]    = prev_date
    rec["PrevInspPCI"]     = prev_pci
    rec["MRDate"]          = mr_date
    rec["MRTreatmentName"] = mr_treat
    rec["LastInspDate"]    = last_date
    rec["LastInspPCI"]     = last_pci
    rec["PCIDifference"]   = pci_diff

    # ---- Calculate diff & determine QC flag ------------------------------
    # Explicitly set QC flag based on meets_criteria
    if pci and pci.get('meets_criteria', False):
        rec["QC"] = "Y"
        print(f"Setting QC=Y for {key}: diff={pci_diff}")
    else:
        rec["QC"] = "N"

    results.append(rec)

print(f"✅ Created records for {len(results)} line features.")
print(f"   • {sum(r['QC']=='Y' for r in results)} meet the QC criteria.")


🔎 Matching attributes with PCI data (all features)…
DEBUG: Found matching criteria record for 1002 - 5466
Setting QC=Y for 1002 - 5466: diff=-14.0
DEBUG: Found matching criteria record for 1004 - 1438
Setting QC=Y for 1004 - 1438: diff=15.0
DEBUG: Found matching criteria record for 1006 - 1287
Setting QC=Y for 1006 - 1287: diff=-18.0
DEBUG: Found matching criteria record for 1014 - 4483
Setting QC=Y for 1014 - 4483: diff=-17.0
DEBUG: Found matching criteria record for 1014 - 5947
Setting QC=Y for 1014 - 5947: diff=-15.0
DEBUG: Found matching criteria record for 1018 - 6575
Setting QC=Y for 1018 - 6575: diff=18.0
DEBUG: Found matching criteria record for 1018 - 2221
Setting QC=Y for 1018 - 2221: diff=16.0
DEBUG: Found matching criteria record for 1018 - 2222
Setting QC=Y for 1018 - 2222: diff=-17.0
DEBUG: Found matching criteria record for 1022 - 1139
Setting QC=Y for 1022 - 1139: diff=-16.0
DEBUG: Found matching criteria record for 1022 - 2727
Setting QC=Y for 1022 - 2727: diff=-14.0


Setting QC=Y for 1212 - 3745: diff=-42.0
DEBUG: Found matching criteria record for 1214 - 4612
Setting QC=Y for 1214 - 4612: diff=-22.0
DEBUG: Found matching criteria record for 1220 - 7386
Setting QC=Y for 1220 - 7386: diff=-13.0
DEBUG: Found matching criteria record for 1230 - 1152
Setting QC=Y for 1230 - 1152: diff=20.0
DEBUG: Found matching criteria record for 1242 - 2168
Setting QC=Y for 1242 - 2168: diff=-21.0
DEBUG: Found matching criteria record for 1242 - 5626
Setting QC=Y for 1242 - 5626: diff=-14.0
DEBUG: Found matching criteria record for 1252 - 5734
Setting QC=Y for 1252 - 5734: diff=-11.0
DEBUG: Found matching criteria record for 1256 - 5595
Setting QC=Y for 1256 - 5595: diff=-55.0
DEBUG: Found matching criteria record for 1266 - 5634
Setting QC=Y for 1266 - 5634: diff=-40.0
DEBUG: Found matching criteria record for 1266 - 5635
Setting QC=Y for 1266 - 5635: diff=-46.0
DEBUG: Found matching criteria record for 1266 - 7305
Setting QC=Y for 1266 - 7305: diff=-22.0
DEBUG: Fou

DEBUG: Found matching criteria record for 1590 - 3421
Setting QC=Y for 1590 - 3421: diff=27.0
DEBUG: Found matching criteria record for 1592 - 1198
Setting QC=Y for 1592 - 1198: diff=-35.0
DEBUG: Found matching criteria record for 1594 - 3396
Setting QC=Y for 1594 - 3396: diff=-52.0
DEBUG: Found matching criteria record for 1602 - 1114
Setting QC=Y for 1602 - 1114: diff=-16.0
DEBUG: Found matching criteria record for 1602 - 5393
Setting QC=Y for 1602 - 5393: diff=-25.0
DEBUG: Found matching criteria record for 1604 - 1235
Setting QC=Y for 1604 - 1235: diff=17.0
DEBUG: Found matching criteria record for 1606 - 1538
Setting QC=Y for 1606 - 1538: diff=-26.0
DEBUG: Found matching criteria record for 1608 - 1095
Setting QC=Y for 1608 - 1095: diff=16.0
DEBUG: Found matching criteria record for 1608 - 3428
Setting QC=Y for 1608 - 3428: diff=16.0
DEBUG: Found matching criteria record for 1610 - 5298
Setting QC=Y for 1610 - 5298: diff=30.0
DEBUG: Found matching criteria record for 1618 - 3811
S

DEBUG: Found matching criteria record for 1884 - 6769
Setting QC=Y for 1884 - 6769: diff=-13.0
DEBUG: Found matching criteria record for 1884 - 6327
Setting QC=Y for 1884 - 6327: diff=-16.0
DEBUG: Found matching criteria record for 1884 - 2035
Setting QC=Y for 1884 - 2035: diff=-13.0
DEBUG: Found matching criteria record for 1888 - 4735
Setting QC=Y for 1888 - 4735: diff=-13.0
DEBUG: Found matching criteria record for 1888 - 4736
Setting QC=Y for 1888 - 4736: diff=-13.0
DEBUG: Found matching criteria record for 1888 - 5534
Setting QC=Y for 1888 - 5534: diff=-11.0
DEBUG: Found matching criteria record for 1900 - 3035
Setting QC=Y for 1900 - 3035: diff=-25.0
DEBUG: Found matching criteria record for 1904 - 2753
Setting QC=Y for 1904 - 2753: diff=-10.0
DEBUG: Found matching criteria record for 1904 - 5450
Setting QC=Y for 1904 - 5450: diff=-15.0
DEBUG: Found matching criteria record for 1908 - 1389
Setting QC=Y for 1908 - 1389: diff=-52.0
DEBUG: Found matching criteria record for 1908 - 1

DEBUG: Found matching criteria record for 2176 - 5753
Setting QC=Y for 2176 - 5753: diff=-16.0
DEBUG: Found matching criteria record for 2176 - 7288
Setting QC=Y for 2176 - 7288: diff=-10.0
DEBUG: Found matching criteria record for 2178 - 1864
Setting QC=Y for 2178 - 1864: diff=-20.0
DEBUG: Found matching criteria record for 2186 - 3813
Setting QC=Y for 2186 - 3813: diff=-19.0
DEBUG: Found matching criteria record for 2186 - 3814
Setting QC=Y for 2186 - 3814: diff=-18.0
DEBUG: Found matching criteria record for 2190 - 1554
Setting QC=Y for 2190 - 1554: diff=-10.0
DEBUG: Found matching criteria record for 2190 - 1733
Setting QC=Y for 2190 - 1733: diff=-14.0
DEBUG: Found matching criteria record for 2192 - 3058
Setting QC=Y for 2192 - 3058: diff=-13.0
DEBUG: Found matching criteria record for 2196 - 6482
Setting QC=Y for 2196 - 6482: diff=-11.0
DEBUG: Found matching criteria record for 2196 - 6478
Setting QC=Y for 2196 - 6478: diff=-10.0
DEBUG: Found matching criteria record for 2196 - 2

DEBUG: Found matching criteria record for 2306 - 7487
Setting QC=Y for 2306 - 7487: diff=-60.0
DEBUG: Found matching criteria record for 2308 - 6051
Setting QC=Y for 2308 - 6051: diff=-31.0
DEBUG: Found matching criteria record for 2316 - 1096
Setting QC=Y for 2316 - 1096: diff=24.0
DEBUG: Found matching criteria record for 2338 - 2731
Setting QC=Y for 2338 - 2731: diff=-37.0
DEBUG: Found matching criteria record for 2338 - 2402
Setting QC=Y for 2338 - 2402: diff=-48.0
DEBUG: Found matching criteria record for 2338 - 2403
Setting QC=Y for 2338 - 2403: diff=-46.0
DEBUG: Found matching criteria record for 2338 - 2404
Setting QC=Y for 2338 - 2404: diff=-56.0
DEBUG: Found matching criteria record for 2342 - 1368
Setting QC=Y for 2342 - 1368: diff=-14.0
DEBUG: Found matching criteria record for 2342 - 7388
Setting QC=Y for 2342 - 7388: diff=-10.0
DEBUG: Found matching criteria record for 2358 - 5769
Setting QC=Y for 2358 - 5769: diff=-10.0
DEBUG: Found matching criteria record for 2365 - 72

Setting QC=Y for 2586 - 3844: diff=41.0
DEBUG: Found matching criteria record for 2588 - 3851
Setting QC=Y for 2588 - 3851: diff=25.0
DEBUG: Found matching criteria record for 2588 - 1820
Setting QC=Y for 2588 - 1820: diff=15.0
DEBUG: Found matching criteria record for 2590 - 5433
Setting QC=Y for 2590 - 5433: diff=37.0
DEBUG: Found matching criteria record for 2592 - 6688
Setting QC=Y for 2592 - 6688: diff=-10.0
DEBUG: Found matching criteria record for 2596 - 7396
Setting QC=Y for 2596 - 7396: diff=-11.0
DEBUG: Found matching criteria record for 2600 - 4629
Setting QC=Y for 2600 - 4629: diff=17.0
DEBUG: Found matching criteria record for 2608 - 3601
Setting QC=Y for 2608 - 3601: diff=27.0
DEBUG: Found matching criteria record for 2614 - 5070
Setting QC=Y for 2614 - 5070: diff=30.0
DEBUG: Found matching criteria record for 2618 - 5707
Setting QC=Y for 2618 - 5707: diff=25.0
DEBUG: Found matching criteria record for 2622 - 1879
Setting QC=Y for 2622 - 1879: diff=-68.0
DEBUG: Found matc

DEBUG: Found matching criteria record for 2856 - 1208
Setting QC=Y for 2856 - 1208: diff=-54.0

In [32]:
# 1. Create the table with specified columns
print(f"Creating table: {table_name}")
arcpy.management.CreateTable(default_gdb, table_name)
arcpy.management.AddField(table_path, "StreetID", "TEXT", field_length=50)
arcpy.management.AddField(table_path, "SectionID", "TEXT", field_length=50)
arcpy.management.AddField(table_path, "StreetName", "TEXT", field_length=100)
arcpy.management.AddField(table_path, "BeginLocation", "TEXT", field_length=100)
arcpy.management.AddField(table_path, "EndLocation", "TEXT", field_length=100)
arcpy.management.AddField(table_path, "PrevInspDate", "DATE")
arcpy.management.AddField(table_path, "PrevInspPCI", "DOUBLE")
arcpy.management.AddField(table_path, "MRDate", "DATE")
arcpy.management.AddField(table_path, "MRTreatmentName", "TEXT", field_length=100)
arcpy.management.AddField(table_path, "LastInspDate", "DATE")
arcpy.management.AddField(table_path, "LastInspPCI", "DOUBLE")
arcpy.management.AddField(table_path, "PCIDifference", "DOUBLE")  # Add PCI Difference field
arcpy.management.AddField(table_path, "Lat", "DOUBLE")
arcpy.management.AddField(table_path, "Long", "DOUBLE")
arcpy.management.AddField(table_path, "MapillaryLink", "TEXT", field_length=255)
arcpy.management.AddField(table_path, "GoogleImageLink", "TEXT", field_length=255)

# Insert the data into the table
print("Inserting data into the table...")
with arcpy.da.InsertCursor(table_path, [
        "StreetID", "SectionID", "StreetName", "BeginLocation", "EndLocation",
        "PrevInspDate", "PrevInspPCI", "MRDate", "MRTreatmentName", 
        "LastInspDate", "LastInspPCI", "PCIDifference", "Lat", "Long", 
        "MapillaryLink", "GoogleImageLink"
    ]) as ic:
    for r in results:
        lat = r['latitude']
        lon = r['longitude']
        
        # Simplified Mapillary link with no filters
        map_url = f"https://www.mapillary.com/app/?lat={lat}&lng={lon}&z=18"
        
        # Google Street View link
        g_url = f"https://www.google.com/maps/@?api=1&map_action=pano&viewpoint={lat},{lon}"
        
        # Get values with defaults for missing fields
        street_id = r.get('street_id', "")
        section_id = r.get('section_id', "")
        street_name = r.get('street_name', "")
        begin_loc = r.get('begin_loc', "")
        end_loc = r.get('end_loc', "")
        prev_insp_date = r.get('prev_insp_date', None)
        prev_insp_pci = r.get('prev_pci', None)
        m_r_date = r.get('m_r_date', None)
        m_r_treatment = r.get('m_r_treatment', "")
        last_insp_date = r.get('last_insp_date', None)
        last_insp_pci = r.get('last_pci', None)
        
        # Calculate PCI difference
        pci_difference = r.get('pci_diff_calc', None)
        
        # Debug output for begin and end locations
        print(f"Record {street_id}-{section_id}: BeginLoc={begin_loc}, EndLoc={end_loc}")
        
        ic.insertRow([
            street_id, section_id, street_name, begin_loc, end_loc,
            prev_insp_date, prev_insp_pci, m_r_date, m_r_treatment,
            last_insp_date, last_insp_pci, pci_difference, lat, lon,
            map_url, g_url
        ])

Inserting data into the table...
Record 1002-5466: BeginLoc=FAIRFIELD ST, EndLoc=SW END
Record 1002-5467: BeginLoc=FAIRFIELD ST, EndLoc=NORTH END
Record 1004-4057: BeginLoc=BRODER ST, EndLoc=BEL AIR ST
Record 1004-5279: BeginLoc=BENWOOD DR, EndLoc=DALE AVE
Record 1004-1438: BeginLoc=BEL AIR ST, EndLoc=BENWOOD DR
Record 1004-3424: BeginLoc=SW END, EndLoc=LAXORE ST
Record 1004-7494: BeginLoc=LAXORE ST, EndLoc=BRODER ST
Record 1006-1287: BeginLoc=CLIFFWOOD AVE, EndLoc=ORANGEWOOD AVE
Record 1008-4045: BeginLoc=COURTRIGHT ST, EndLoc=EAST END
Record 1010-6775: BeginLoc=BANYAN LN, EndLoc=NORTH END
Record 1012-5159: BeginLoc=SOUTH END, EndLoc=MANCHESTER AVE
Record 1014-4483: BeginLoc=SOUTH END, EndLoc=BROADWAY
Record 1014-5947: BeginLoc=MABLE ST, EndLoc=BROADWAY
Record 1016-6075: BeginLoc=ADDINGTON DR, EndLoc=NE END
Record 1018-6573: BeginLoc=PAGEANT ST, EndLoc=REDROCK ST
Record 1018-6574: BeginLoc=FINCH ST, EndLoc=ADDINGTON CIR
Record 1018-6575: BeginLoc=BAINBRIDGE AVE, EndLoc=FAUNA AVE
Recor

Record 1130-1617: BeginLoc=WOODSBORO AVE, EndLoc=NORTH END
Record 1132-7083: BeginLoc=LIBERTY AVE, EndLoc=FREEDOM AVE
Record 1134-6325: BeginLoc=MARJAN ST, EndLoc=DUNE ST
Record 1134-2857: BeginLoc=DUNE ST, EndLoc=RIO VISTA ST
Record 1134-2858: BeginLoc=CARDIFF ST, EndLoc=MARJAN ST
Record 1136-1936: BeginLoc=AMES AVE CDS, EndLoc=NORTH END
Record 1136-1937: BeginLoc=AMES AVE CDS, EndLoc=NORTH END
Record 1136-107127: BeginLoc=AMES AVE CDS, EndLoc=AMES AVE CDS
Record 1136-207127: BeginLoc=AMES AVE CDS, EndLoc=SUNKIST ST
Record 1136-7127: BeginLoc=HILDA ST, EndLoc=AMES AVE CDS
Record 1138-2865: BeginLoc=HILDA ST, EndLoc=EAST END
Record 1140-5794: BeginLoc=CARNEGIE AVE, EndLoc=NW END
Record 1142-5164: BeginLoc=CRONE AVE, EndLoc=DAMON AVE
Record 1141-5771: BeginLoc=NOHL RANCH RD, EndLoc=SILVERSPUR TR
Record 1141-6282: BeginLoc=CAMINO CORRER, EndLoc=SANTA ANA CANYON RD WB
Record 1144-7014: BeginLoc=ANCHOR ST, EndLoc=LEWIS ST
Record 1144-3991: BeginLoc=ANCHOR ST, EndLoc=NORTH END
Record 1144-7

Record 1270-7340: BeginLoc=CAMINO CORRER, EndLoc=PASEO RIO AZUL
Record 1270-7341: BeginLoc=CALLE CEDRO, EndLoc=CAMINO CORRER
Record 1272-2090: BeginLoc=CAMINO CORRER, EndLoc=AVENIDA ARBOL
Record 1272-2091: BeginLoc=CALLE CEDRO, EndLoc=CAMINO CORRER
Record 1274-1874: BeginLoc=CALLE DA GAMA, EndLoc=VIA CORTEZ
Record 1274-2544: BeginLoc=VIA CORTEZ, EndLoc=CALLE DIAZ
Record 1276-6372: BeginLoc=CALLE GRANADA, EndLoc=CAMINO TAMPICO
Record 1276-5756: BeginLoc=PASEO LAREDO, EndLoc=CALLE DURANGO
Record 1276-5757: BeginLoc=CAMINO TAMPICO, EndLoc=PASEO LAREDO
Record 1278-1328: BeginLoc=CALLE GRANADA, EndLoc=CALLE DURANGO
Record 1280-2771: BeginLoc=VIA ALISTA, EndLoc=CALLE DURANGO
Record 1280-5759: BeginLoc=PASEO TAMPICO, EndLoc=PASEO LAREDO
Record 1280-6371: BeginLoc=PASEO TAMPICO, EndLoc=AVENIDA JUAREZ
Record 1280-5469: BeginLoc=PASEO LAREDO, EndLoc=VIA ALISTA
Record 1282-7344: BeginLoc=AVENIDA ARBOL, EndLoc=PASEO RIO AZUL
Record 1284-5572: BeginLoc=GARY PL, EndLoc=HUKEE AVE
Record 1284-5573: Be

Record 1400-1774: BeginLoc=RADCLIFFE AVE, EndLoc=NORTH END
Record 1402-7379: BeginLoc=CANYON VISTA DR, EndLoc=NORTH END
Record 1404-1193: BeginLoc=BROADWAY, EndLoc=FRONTAGE RD
Record 1406-2994: BeginLoc=TYLER AVE, EndLoc=MONROE AVE
Record 1406-2995: BeginLoc=MONROE AVE, EndLoc=COOLIDGE AVE
Record 1406-6983: BeginLoc=LINCOLN AVE, EndLoc=POLK AVE
Record 1406-6984: BeginLoc=POLK AVE, EndLoc=TYLER AVE
Record 1406-7504: BeginLoc=CITY BOUNDARY, EndLoc=COOLIDGE AVE
Record 1408-5062: BeginLoc=BRIDGEPORT AVE, EndLoc=ORANGE AVE
Record 1408-3347: BeginLoc=ACADEMY AVE, EndLoc=NORTH END
Record 1408-3418: BeginLoc=SKYWOOD CIR, EndLoc=ACADEMY AVE
Record 1408-3419: BeginLoc=ROWLAND CIR, EndLoc=SKYWOOD CIR
Record 1408-3420: BeginLoc=BRIDGEPORT AVE, EndLoc=ROWLAND CIR
Record 1410-1349: BeginLoc=OMEGA AVE, EndLoc=BALL RD
Record 1410-5683: BeginLoc=BALL RD, EndLoc=ALMONT AVE
Record 1412-3276: BeginLoc=LINCOLN AVE, EndLoc=NORTH END
Record 1414-3164: BeginLoc=CLIFPARK CIR, EndLoc=DOVER CIR
Record 1414-7084:

Record 1546-5845: BeginLoc=VIA DE ROSA, EndLoc=NORTH END
Record 1548-1504: BeginLoc=FREMONT ST, EndLoc=ORIOLE ST
Record 1550-5443: BeginLoc=KINGS COURT DR, EndLoc=EAST END
Record 1552-1123: BeginLoc=SOUTH ST, EndLoc=ELDERWOOD AVE
Record 1554-5542: BeginLoc=PALM ST, EndLoc=HICKORY ST
Record 1556-1440: BeginLoc=GREENTREE CIR, EndLoc=THORNTON AVE
Record 1558-1154: BeginLoc=HOUSTON AVE, EndLoc=AVONDALE PL
Record 1558-3266: BeginLoc=HOUSTON AVE, EndLoc=PICADILLY WAY
Record 1560-5949: BeginLoc=LOARA ST, EndLoc=SONYA PL
Record 1560-4508: BeginLoc=GILMAR ST, EndLoc=GILBUCK DR
Record 1560-4509: BeginLoc=WILDE PL, EndLoc=GILMAR ST
Record 1560-4510: BeginLoc=SONYA PL, EndLoc=WILDE PL
Record 1562-4504: BeginLoc=ALVY ST, EndLoc=EAST END
Record 1562-4505: BeginLoc=ALVY ST, EndLoc=ALVY ST
Record 1564-7021: BeginLoc=SANDALWOOD AVE, EndLoc=BANYAN AVE
Record 1564-7022: BeginLoc=BANYAN AVE, EndLoc=WARRENTON AVE
Record 1564-3019: BeginLoc=BELMONT AVE, EndLoc=SANDALWOOD AVE
Record 1566-1640: BeginLoc=LA PA

...

Record 1944-7003: BeginLoc=GLENCREST AVE, EndLoc=GRAMERCY AVE
Record 1946-1613: BeginLoc=LA PALMA AVE, EndLoc=NORTH END
Record 1948-5979: BeginLoc=MOONSTONE ST, EndLoc=EAST END
Record 1950-6547: BeginLoc=NOHL CANYON RD, EndLoc=EAST END
Record 1952-2542: BeginLoc=CALLE DIAZ, EndLoc=SOLOMON DR
Record 1954-1644: BeginLoc=VIA RIBAZO, EndLoc=CIRCULO ROBEL
Record 1956-6401: BeginLoc=VIA MONTANERA, EndLoc=NE END
Record 1958-2411: BeginLoc=PURITAN PL, EndLoc=NORTH END
Record 1958-2412: BeginLoc=VIRGINIA AVE, EndLoc=PURITAN PL
Record 1960-4833: BeginLoc=STANDISH AVE, EndLoc=ALDEN PL
Record 1962-2416: BeginLoc=DIANA AVE, EndLoc=SOUTH ST
Record 1964-4921: BeginLoc=VERDE AVE, EndLoc=NORTH END
Record 1964-4922: BeginLoc=RANDY AVE, EndLoc=VERDE AVE
Record 1964-4923: BeginLoc=BURNTWOOD AVE, EndLoc=RANDY AVE
Record 1966-5178: BeginLoc=EUCLID ST, EndLoc=CINDY PL
Record 1968-3769: BeginLoc=NORTH END, EndLoc=CINDY LN
Record 1970-6550: BeginLoc=WEST END, EndLoc=NOHL CANYON RD
Record 1972-7282: BeginLoc=CI

Record 3000A-1000: BeginLoc=RUNNING SPRINGS DR, EndLoc=CRESTVIEW LN
Record 2056-3722: BeginLoc=MAPLE ST, EndLoc=HOLLY ST
Record 2056-3723: BeginLoc=COLUMBINE PL, EndLoc=MAPLE ST
Record 2056-4263: BeginLoc=LOTUS PL, EndLoc=JASMINE PL
Record 2056-4264: BeginLoc=CATALPA AVE, EndLoc=LOTUS PL
Record 2056-6036: BeginLoc=JASMINE PL, EndLoc=COLUMBINE PL
Record 2058-4262: BeginLoc=GROTON ST, EndLoc=FULTON ST
Record 2060-5349: BeginLoc=ALAMO ST, EndLoc=MONTEREY ST
Record 2062-7377: BeginLoc=BANNER RIDGE DR, EndLoc=GARDEN VIEW DR
Record 2064-1342: BeginLoc=CENTER ST, EndLoc=CYPRESS ST
Record 2064-2627: BeginLoc=CYPRESS ST, EndLoc=NORTH END
Record 2066-1229: BeginLoc=COLONY ST, EndLoc=CITY BOUNDARY
Record 2066-5367: BeginLoc=COLONY ST, EndLoc=BROOKHURST ST
Record 2068-2989: BeginLoc=STOCKTON AVE, EndLoc=CRESCENT AVE
Record 2070-2990: BeginLoc=RUSSELL PL, EndLoc=MONROE AVE
Record 2070-2991: BeginLoc=RUSSELL PL, EndLoc=STANLEY PL
Record 2070-2992: BeginLoc=STANLEY PL, EndLoc=CORNELL PL
Record 2070-2

Record 4498-6622: BeginLoc=WEST END, EndLoc=TABLEROCK PL
Record 4498-6439: BeginLoc=PANORAMA DR, EndLoc=EAST END
Record 4500-6382: BeginLoc=HACKAMORE LN, EndLoc=SHANADA ST
Record 4502-6608: BeginLoc=MAL CT, EndLoc=TREVOR ST
Record 4502-5696: BeginLoc=BELVEDERE RD, EndLoc=GREENHEDGE AVE
Record 4502-1790: BeginLoc=GREENHEDGE AVE, EndLoc=MAL CT
Record 4504-7241: BeginLoc=QUIET CANYON LN, EndLoc=EAST END
Record 4504-6124: BeginLoc=WEST END, EndLoc=QUIET CANYON LN
Record 4506-5858: BeginLoc=QUIET CANYON CT, EndLoc=OAK CANYON DR
Record 4508-4664: BeginLoc=SOUTH END, EndLoc=CARNEGIE AVE
Record 4510-4720: BeginLoc=BAJA DR, EndLoc=RIO GRANDE DR
Record 4510-5767: BeginLoc=ARBORETUM RD, EndLoc=PALO ALTO DR
Record 4510-4924: BeginLoc=PALO ALTO DR, EndLoc=BAJA DR
Record 4510-6369: BeginLoc=SANTA ANA CANYON RD, EndLoc=CAMINO CORRER
Record 4510-1329: BeginLoc=RIO GRANDE DR, EndLoc=SANTA ANA CANYON RD
Record 4512-1025: BeginLoc=RADCLIFFE AVE @WYE, EndLoc=ORANGETHORPE AVE
Record 4512-1775: BeginLoc=KEN

Record 4634-7475: BeginLoc=SMOKEWOOD CIR, EndLoc=BURLWOOD DR
Record 4636-2059: BeginLoc=DONNA CT, EndLoc=FAIRMONT BLVD
Record 4636-6363: BeginLoc=CANYON WOODS RD, EndLoc=AVENIDA LYSANNA
Record 4636-6364: BeginLoc=AVENIDA LYSANNA, EndLoc=QUINTANA DR
Record 4636-6700: BeginLoc=DONNA CT, EndLoc=CANYON WOODS RD
Record 4638-5101: BeginLoc=SOUTH END, EndLoc=CERRITOS AVE
Record 4639-3686: BeginLoc=CERRITOS AVE, EndLoc=LULLABY LN
Record 4640-7179: BeginLoc=MOUNTAIN LOOP TR, EndLoc=IMPERIAL HWY
Record 5706-7528: BeginLoc=ROXANNE, EndLoc=REDROCK
Record 4642-5846: BeginLoc=WEST END, EndLoc=SANTA ANA CANYON RD
Record 4642-4957: BeginLoc=SANTA ANA CANYON RD, EndLoc=AMBERWOOD ST
Record 4644-6365: BeginLoc=ARBORETUM RD, EndLoc=SE END
Record 4646-1630: BeginLoc=NORTH END, EndLoc=ROMNEYA DR
Record 4648-6166: BeginLoc=COLUMBUS DR, EndLoc=SE END
Record 4650-3216: BeginLoc=RAINBOW AVE, EndLoc=NORTH END
Record 4652-3208: BeginLoc=GRAMERCY AVE, EndLoc=GREENACRE AVE
Record 4652-3209: BeginLoc=GLENCREST AVE, 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
# ================================================================
# Cell 13 – Create & populate QC_Points feature class
# ================================================================

print(f"\nCreating points feature class in WGS84: {fc_name}")
arcpy.management.CreateFeatureclass(
    default_gdb, fc_name, "POINT", spatial_reference=wgs84
)

# ---- Add fields -------------------------------------------------
field_defs = [
    ("StreetID",          "TEXT",   50),
    ("SectionID",         "TEXT",   50),
    ("StreetName",        "TEXT",  100),
    ("BeginLocation",     "TEXT",  100),
    ("EndLocation",       "TEXT",  100),
    ("PrevInspDate",      "DATE"),
    ("PrevInspPCI",       "DOUBLE"),
    ("MRDate",            "DATE"),
    ("MRTreatmentName",   "TEXT",  100),
    ("LastInspDate",      "DATE"),
    ("LastInspPCI",       "DOUBLE"),
    ("PCIDifference",     "DOUBLE"),
    ("Lat",               "DOUBLE"),
    ("Long",              "DOUBLE"),
    ("MapillaryLink",     "TEXT", 255),
    ("GoogleImageLink",   "TEXT", 255),
    ("QC",                "TEXT",   1)   # <‑‑ NEW
]
for name, ftype, *rest in field_defs:
    length = rest[0] if rest else None
    arcpy.management.AddField(fc_path, name, ftype, field_length=length)

# ---- Insert rows ------------------------------------------------
print("Populating the points feature class…")
with arcpy.da.InsertCursor(
        fc_path,
        ["SHAPE@", "StreetID", "SectionID", "StreetName", "BeginLocation",
         "EndLocation", "PrevInspDate", "PrevInspPCI", "MRDate",
         "MRTreatmentName", "LastInspDate", "LastInspPCI",
         "PCIDifference", "Lat", "Long", "MapillaryLink",
         "GoogleImageLink", "QC"]        # <‑‑ QC added
) as curs:
    for r in results:
        pt   = arcpy.Point(r["longitude"], r["latitude"])
        geom = arcpy.PointGeometry(pt, wgs84)

        lat, lon  = r["latitude"], r["longitude"]
        map_link  = f"https://www.mapillary.com/app/?lat={lat}&lng={lon}&z=18"
        gsv_link  = f"https://www.google.com/maps/@?api=1&map_action=pano&viewpoint={lat},{lon}"

        curs.insertRow([
            geom,
            r.get("street_id"), r.get("section_id"), r.get("street_name"),
            r.get("begin_loc"), r.get("end_loc"),
            r.get("PrevInspDate"), r.get("PrevInspPCI"), r.get("MRDate"),
            r.get("MRTreatmentName"), r.get("LastInspDate"),
            r.get("LastInspPCI"), r.get("PCIDifference"),
            lat, lon, map_link, gsv_link, r["QC"]     # <‑‑ QC value
        ])

print("✅ Points created & attributes populated.")


Populating the points feature class…
✅ Points created & attributes populated.


In [34]:
# Define the add_layer_and_style function first
def add_layer_and_style(layer_path, layer_type="POINT", color=None, size=None, use_tear_pin=False):
    """
    Add a layer to the map only if it doesn't already exist, and apply styling.
    Added support for Tear Pin symbology for points.
    Returns the layer if it was added, None otherwise.
    """
    print(f"Adding layer: {os.path.basename(layer_path)}")
    
    # Check if layer already exists in the map
    existing_layer = None
    for lyr in active_map.listLayers():
        try:
            if lyr.dataSource == layer_path:
                existing_layer = lyr
                print(f"Layer already exists in map: {lyr.name}")
                break
        except:
            continue
    
    # If layer doesn't exist, add it
    if not existing_layer:
        active_map.addDataFromPath(layer_path)
        
        # Find the newly added layer
        for lyr in active_map.listLayers():
            try:
                if lyr.dataSource == layer_path:
                    existing_layer = lyr
                    print(f"Added layer to map: {lyr.name}")
                    break
            except:
                continue
    
    # Style the layer if found
    if existing_layer:
        if layer_type == "POINT" and use_tear_pin:
            try:
                # Attempt to set the Tear Pin symbol
                print("Attempting to apply Tear Pin symbol...")
                
                # Method 1: Try using the symbology classes
                try:
                    symb = existing_layer.symbology
                    
                    # Step 1: Get the current symbol
                    if hasattr(symb, 'renderer') and hasattr(symb.renderer, 'symbol'):
                        # Try to change the symbol style
                        try:
                            # Change the symbol type to a marker symbol
                            symb.renderer.symbol.style = "Tear Pin 1"
                            
                            # Set the size if provided
                            if size:
                                symb.renderer.symbol.size = size
                            else:
                                symb.renderer.symbol.size = 12  # Default tear pin size
                                
                            # Set the color if provided
                            if color:
                                symb.renderer.symbol.color = {'RGB': color}
                                
                            # Apply the symbology
                            existing_layer.symbology = symb
                            print("Applied Tear Pin symbol to point layer")
                        except Exception as e:
                            print(f"Could not set Tear Pin style: {str(e)}")
                            
                            # Try alternative method - just set size and color
                            try:
                                if size:
                                    symb.renderer.symbol.size = size
                                if color:
                                    symb.renderer.symbol.color = {'RGB': color}
                                existing_layer.symbology = symb
                                print("Applied basic styling (size/color) to point layer")
                                print("Please manually change the symbol to Tear Pin 1:")
                                print("1. Right-click the layer → Symbology")
                                print("2. Click on the symbol")
                                print("3. In the Format Point Symbol pane, select 'Tear Pin 1' from the gallery")
                            except:
                                print("Could not apply even basic styling")
                except Exception as e:
                    print(f"Could not modify symbol: {str(e)}")
                    print("Please manually set the Tear Pin symbol:")
                    print("1. Right-click the point layer in the Contents pane")
                    print("2. Select 'Symbology'")
                    print("3. Click on the current symbol to open the symbol selector")
                    print("4. Choose 'Tear Pin 1' from the gallery")
                    print("5. Set the appropriate size and color")
            except Exception as e:
                print(f"Could not apply Tear Pin styling: {str(e)}")
        elif color:  # Regular styling for other layer types or when not using Tear Pin
            try:
                symb = existing_layer.symbology
                if hasattr(symb, 'renderer'):
                    if hasattr(symb.renderer, 'symbol'):
                        if color:
                            symb.renderer.symbol.color = {'RGB': color}
                        if size and layer_type == "POINT":
                            symb.renderer.symbol.size = size
                        existing_layer.symbology = symb
                        print(f"Applied styling to layer: {existing_layer.name}")
            except Exception as e:
                print(f"Could not apply styling: {str(e)}")
    
    return existing_layer

# Now use the function to add layers only if needed, with proper styling
print("\nAdding outputs to the map (avoiding duplicates)...")

# Add point features with red styling
point_layer = add_layer_and_style(fc_path, "POINT", [255, 0, 0, 100], 8)

# Add table (tables don't need styling)
print("Adding table to map...")
existing_table = None
for tbl in active_map.listTables():
    try:
        if tbl.dataSource == table_path:
            existing_table = tbl
            print(f"Table already exists in map: {tbl.name}")
            break
    except:
        continue

if not existing_table:
    active_map.addDataFromPath(table_path)
    print(f"Added table to map: {table_name}")

# Save the project to preserve changes
aprx.save()
print("Project saved successfully with all changes.")
print("\nScript completed successfully!")
print(f"Created table: {table_name}")
print(f"Created feature class: {fc_name}")
print(f"Found {len(results)} points that met the threshold criteria.")

# Summary report of Begin/End Location fields processing
print("\n===== Begin/End Location Fields Summary =====")
begin_loc_feature_found = "Yes" if begin_loc_field else "No"
end_loc_feature_found = "Yes" if end_loc_field else "No"
begin_loc_pci_found = "Yes" if begin_loc_field_pci else "No"
end_loc_pci_found = "Yes" if end_loc_field_pci else "No"

print(f"Begin Location field found in feature layer: {begin_loc_feature_found}")
print(f"End Location field found in feature layer: {end_loc_feature_found}")
print(f"Begin Location field found in PCI table: {begin_loc_pci_found}")
print(f"End Location field found in PCI table: {end_loc_pci_found}")
print("============================================")


Adding outputs to the map (avoiding duplicates)...
Adding layer: QC_Points
Layer already exists in map: QC_Points
Applied styling to layer: QC_Points
Adding table to map...
Table already exists in map: QC_Table
Project saved successfully with all changes.

Script completed successfully!
Created table: QC_Table
Created feature class: QC_Points
Found 6334 points that met the threshold criteria.

===== Begin/End Location Fields Summary =====
Begin Location field found in feature layer: Yes
End Location field found in feature layer: Yes
Begin Location field found in PCI table: Yes
End Location field found in PCI table: Yes


In [35]:
# This cell is optional - you can create a wrapper function to call everything
def find_midpoints_with_pci_matching():
    """Main function to execute the workflow"""
    try:
        # All the code from cells 3-15 would go here
        print("Completed successfully!")
    except Exception as e:
        import traceback
        print(f"ERROR: An exception occurred during script execution:\n{str(e)}")
        print(traceback.format_exc())
        
        # Try to save any incomplete but important data
        try:
            if 'aprx' in locals():
                aprx.save()
                print("Project saved despite error.")
        except:
            print("Could not save project after error.")

# Uncomment the line below if you want to run everything at once
# find_midpoints_with_pci_matching()

In [36]:
# ---------------------------------------------------------------
# Spatial‑join back to the original line feature with QC field at the end
# ---------------------------------------------------------------
print("\n===== Performing Spatial Join to Original Line Feature =====")

# Verify that the paths exist before proceeding
print("Verifying paths...")
print(f"PMP_Path: {PMP_Path}")
print(f"fc_path: {fc_path}")

if not arcpy.Exists(PMP_Path):
    print(f"ERROR: Target feature class does not exist at {PMP_Path}")
    # Try to find the feature class in the current project
    print("Attempting to locate the feature class in the current project...")
    
    # Get the active map and find the feature layer again
    PMP_Path = None
    active_map = None
    
    for m in aprx.listMaps():
        for lyr in m.listLayers():
            if lyr.isFeatureLayer and "SSstreet" in lyr.name:
                PMP_Path = lyr.dataSource
                wildomar_layer = lyr
                active_map = m
                print(f"Found layer: {lyr.name} in map: {m.name}")
                print(f"Using data source: {PMP_Path}")
                break
        if PMP_Path:
            break
    
    if not PMP_Path:
        print("ERROR: Could not find the feature class in the project.")
        print("Unable to proceed with spatial join.")
        # Exit cell execution gracefully
        raise ValueError("Target feature class not found")

if not arcpy.Exists(fc_path):
    print(f"ERROR: Points feature class does not exist at {fc_path}")
    print("Unable to proceed with spatial join.")
    # Exit cell execution gracefully
    raise ValueError("Points feature class not found")

# 1) Field mapping ------------------------------------------------
print("Setting up field mapping...")

# First, get all fields from the target feature class except QC
try:
    target_fields = [f.name for f in arcpy.ListFields(PMP_Path) if f.name.upper() != "QC" and not f.name.upper().startswith("QC_")]
    print(f"Found {len(target_fields)} fields in target feature class")
except Exception as e:
    print(f"ERROR listing fields from target: {str(e)}")
    print("Using layer object directly instead of path...")
    # Try using the layer object directly
    target_fields = [f.name for f in arcpy.ListFields(wildomar_layer) if f.name.upper() != "QC" and not f.name.upper().startswith("QC_")]
    print(f"Found {len(target_fields)} fields in target layer")

# Create a FieldMappings object for the spatial join
field_mapping = arcpy.FieldMappings()

# Add all fields from the target feature class first (except QC)
for field_name in target_fields:
    if field_name not in ["OBJECTID", "SHAPE", "Shape", "SHAPE_Length", "Shape_Length"]:
        try:
            # Create a field map for each field
            field_map = arcpy.FieldMap()
            
            # Try using PMP_Path first, fall back to wildomar_layer if needed
            try:
                field_map.addInputField(PMP_Path, field_name)
            except:
                field_map.addInputField(wildomar_layer, field_name)
                
            # Add the field map to the mappings
            field_mapping.addFieldMap(field_map)
        except Exception as e:
            print(f"Warning: Could not add field {field_name}: {str(e)}")

# Now add selected fields from the points feature class
selected_fields = [
    {"name": "PrevInspDate",    "alias": "Previous_Date"},
    {"name": "PrevInspPCI",     "alias": "Previous_PCI"},
    {"name": "MRDate",          "alias": "MR_Date"},
    {"name": "MRTreatmentName", "alias": "MR_Treatment"},
    {"name": "LastInspDate",    "alias": "Last_Insp_Date"},
    {"name": "LastInspPCI",     "alias": "Last_Insp_PCI"},
    {"name": "PCIDifference",   "alias": "PCI_Diff"},
    {"name": "MapillaryLink",   "alias": "Mapillary"},
    {"name": "GoogleImageLink", "alias": "Google"},
]

# Add each field from the points to the mapping
for fld in selected_fields:
    try:
        # Create a field map for this field
        fm = arcpy.FieldMap()
        
        # Add the input field from the points feature class
        fm.addInputField(fc_path, fld["name"])
        
        # Set the output field properties
        out_fld = fm.outputField
        out_fld.name = fld["alias"]
        out_fld.aliasName = fld["alias"]
        fm.outputField = out_fld
        
        # Add the field map to the mappings
        field_mapping.addFieldMap(fm)
    except Exception as e:
        print(f"Warning: Could not add field {fld['name']}: {str(e)}")

# Finally, add the QC field LAST to ensure it appears at the end
try:
    qc_field_map = arcpy.FieldMap()
    qc_field_map.addInputField(fc_path, "QC")
    out_qc_field = qc_field_map.outputField
    out_qc_field.name = "QC"
    out_qc_field.aliasName = "QC"
    qc_field_map.outputField = out_qc_field
    field_mapping.addFieldMap(qc_field_map)
    print(f"Added QC field as the last field")
except Exception as e:
    print(f"Warning: Could not add QC field: {str(e)}")

print(f"Field mapping set up with QC field positioned last")

# 2) Run the spatial join ---------------------------------------
print(f"Creating joined feature class: {joined_fc_name}")

# First delete the output if it already exists
if arcpy.Exists(joined_fc_path):
    arcpy.management.Delete(joined_fc_path)
    print(f"Deleted existing {joined_fc_name}")

# Execute the spatial join - try with direct layer objects if paths fail
try:
    # Try using the paths
    arcpy.analysis.SpatialJoin(
        target_features=PMP_Path,
        join_features=fc_path,
        out_feature_class=joined_fc_path,
        join_operation="JOIN_ONE_TO_ONE",
        join_type="KEEP_ALL",
        field_mapping=field_mapping,
        match_option="CLOSEST",
        search_radius="10 Meters"
    )
    print("Spatial join completed using paths")
except Exception as e:
    print(f"Error using paths for spatial join: {str(e)}")
    print("Trying with layer objects...")
    
    try:
        # Try using the layer objects directly
        arcpy.analysis.SpatialJoin(
            target_features=wildomar_layer,
            join_features=active_map.listLayers(fc_name)[0] if fc_name in [lyr.name for lyr in active_map.listLayers()] else fc_path,
            out_feature_class=joined_fc_path,
            join_operation="JOIN_ONE_TO_ONE",
            join_type="KEEP_ALL",
            field_mapping=field_mapping,
            match_option="CLOSEST",
            search_radius="10 Meters"
        )
        print("Spatial join completed using layer objects")
    except Exception as e2:
        print(f"Error with layer objects as well: {str(e2)}")
        print("Unable to complete spatial join")
        raise e2

# 3) Verify QC field is populated correctly ---------------------
if arcpy.Exists(joined_fc_path):
    print("Verifying QC field in joined shapefile...")

    # First, check if QC or QC_1 field exists
    field_names = [f.name for f in arcpy.ListFields(joined_fc_path)]
    has_qc = "QC" in field_names
    has_qc_1 = "QC_1" in field_names

    print(f"Fields in joined shapefile:")
    print(f"QC field exists: {has_qc}")
    print(f"QC_1 field exists: {has_qc_1}")

    # Determine which QC field to use
    use_qc_field = "QC" if has_qc else ("QC_1" if has_qc_1 else None)

    if use_qc_field:
        print(f"Using {use_qc_field} field. Checking values...")
        
        # Check QC field values
        y_count = 0
        n_count = 0
        null_count = 0
        
        with arcpy.da.SearchCursor(joined_fc_path, [use_qc_field]) as cursor:
            for row in cursor:
                if row[0] == "Y":
                    y_count += 1
                elif row[0] == "N":
                    n_count += 1
                else:
                    null_count += 1
        
        print(f"QC field values: Y={y_count}, N={n_count}, Null={null_count}")
        
        # If QC values aren't populated correctly, update them
        if y_count == 0 or null_count > 0:
            print("QC field not populated correctly. Running update...")
            
            # First, set all to "N"
            with arcpy.da.UpdateCursor(joined_fc_path, [use_qc_field]) as cursor:
                for row in cursor:
                    row[0] = "N"
                    cursor.updateRow(row)
            
            # Then set appropriate records to "Y"
            y_keys = []
            with arcpy.da.SearchCursor(fc_path, ["StreetID", "SectionID", "QC"]) as cursor:
                for row in cursor:
                    if row[2] == "Y":
                        street_id = str(row[0]).strip() if row[0] else ""
                        section_id = str(row[1]).strip() if row[1] else ""
                        key = f"{street_id} - {section_id}"
                        y_keys.append(key)
            
            print(f"Found {len(y_keys)} records with QC=Y in points layer")
            
            updated_count = 0
            with arcpy.da.UpdateCursor(joined_fc_path, ["StreetID", "SectionID", use_qc_field]) as cursor:
                for row in cursor:
                    street_id = str(row[0]).strip() if row[0] else ""
                    section_id = str(row[1]).strip() if row[1] else ""
                    key = f"{street_id} - {section_id}"
                    
                    if key in y_keys:
                        row[2] = "Y"
                        cursor.updateRow(row)
                        updated_count += 1
            
            print(f"Updated {updated_count} records to {use_qc_field}=Y")
    else:
        print("ERROR: No QC field found in joined shapefile!")
else:
    print("ERROR: Joined shapefile was not created successfully")

print("✅ Spatial join process completed")


===== Performing Spatial Join to Original Line Feature =====
Verifying paths...
PMP_Path: C:\Users\JLin\Documents\ArcGIS\Packages\QC Tool_e987fe\commondata\ssstreet_4\SSstreet
fc_path: C:\Users\JLin\Downloads\QC Tool\Default.gdb\QC_Points
ERROR: Target feature class does not exist at C:\Users\JLin\Documents\ArcGIS\Packages\QC Tool_e987fe\commondata\ssstreet_4\SSstreet
Attempting to locate the feature class in the current project...
Found layer: SSstreet in map: Map
Using data source: C:\Users\JLin\Documents\ArcGIS\Packages\QC Tool_e987fe\commondata\ssstreet_4\SSstreet
Setting up field mapping...
ERROR listing fields from target: "C:\Users\JLin\Documents\ArcGIS\Packages\QC Tool_e987fe\commondata\ssstreet_4\SSstreet" does not exist
Using layer object directly instead of path...
Found 28 fields in target layer
Added QC field as the last field
Field mapping set up with QC field positioned last
Creating joined feature class: QC_Joined_Shapefile
Deleted existing QC_Joined_Shapefile
Spatial 

In [37]:
# ---------------------------------------------------------------
# Fix for QC Fields in QC_Joined_Shapefile - CORRECTED KEY FORMAT
# ---------------------------------------------------------------
print("\n===== FIXING QC FIELDS - CORRECTED KEY FORMAT =====")

# Check if the joined feature class exists
if not arcpy.Exists(joined_fc_path):
    print(f"ERROR: {joined_fc_path} does not exist!")
else:
    # Verify QC field exists
    field_names = [f.name for f in arcpy.ListFields(joined_fc_path)]
    has_qc = "QC" in field_names
    
    if not has_qc:
        print("QC field does not exist. Adding it...")
        arcpy.management.AddField(joined_fc_path, "QC", "TEXT", field_length=1)
    
    # Create a list of keys (Street_ID - Section_ID) that should be "Y"
    y_keys = []
    
    # DEBUG: Check the format of values in the points layer
    print("Examining the first 5 QC=Y records in the points layer:")
    sample_count = 0
    
    with arcpy.da.SearchCursor(fc_path, ["StreetID", "SectionID", "QC"]) as cursor:
        for row in cursor:
            if row[2] == "Y":
                # Extract the IDs, correcting the duplicated format if needed
                street_id_raw = str(row[0]).strip() if row[0] else ""
                section_id_raw = str(row[1]).strip() if row[1] else ""
                
                # Check if the street_id contains a duplication pattern
                if " - " in street_id_raw:
                    parts = street_id_raw.split(" - ")
                    if len(parts) >= 2:
                        street_id = parts[0].strip()
                        # If section_id also has this pattern, use the corrected part
                        if " - " in section_id_raw:
                            section_parts = section_id_raw.split(" - ")
                            if len(section_parts) >= 2:
                                section_id = section_parts[1].strip()
                            else:
                                section_id = section_id_raw
                        else:
                            section_id = section_id_raw
                    else:
                        street_id = street_id_raw
                        section_id = section_id_raw
                else:
                    street_id = street_id_raw
                    section_id = section_id_raw
                
                # Create the properly formatted key
                key = f"{street_id} - {section_id}"
                
                # Print DEBUG info
                if sample_count < 5:
                    print(f"Raw values: StreetID='{street_id_raw}', SectionID='{section_id_raw}'")
                    print(f"Corrected key: '{key}'")
                    sample_count += 1
                
                y_keys.append(key)
    
    print(f"Found {len(y_keys)} records with QC=Y in points layer")
    
    # Now let's look at how the values appear in the joined shapefile
    print("\nSampling keys from joined shapefile:")
    sample_count = 0
    
    with arcpy.da.SearchCursor(joined_fc_path, ["StreetID", "SectionID"]) as cursor:
        for row in cursor:
            street_id = str(row[0]).strip() if row[0] else ""
            section_id = str(row[1]).strip() if row[1] else ""
            key = f"{street_id} - {section_id}"
            
            if sample_count < 5:
                print(f"Joined shapefile key: '{key}'")
                print(f"Is this key in Y keys? {key in y_keys}")
                sample_count += 1
    
    # Update the joined feature class without setting everything to "N" first
    print("\nUpdating QC values in joined shapefile...")
    updated_count = 0
    
    with arcpy.da.UpdateCursor(joined_fc_path, ["StreetID", "SectionID", "QC"]) as cursor:
        for row in cursor:
            street_id = str(row[0]).strip() if row[0] else ""
            section_id = str(row[1]).strip() if row[1] else ""
            key = f"{street_id} - {section_id}"
            
            # Only update if the key is in the Y keys list
            if key in y_keys:
                row[2] = "Y"
                cursor.updateRow(row)
                updated_count += 1
                
                # Debug output for first few updates
                if updated_count <= 5:
                    print(f"Updated key: {key}")
    
    print(f"Updated {updated_count} records to QC=Y")
    
    # Final verification
    y_count = 0
    n_count = 0
    null_count = 0
    
    with arcpy.da.SearchCursor(joined_fc_path, ["QC"]) as cursor:
        for row in cursor:
            if row[0] == "Y":
                y_count += 1
            elif row[0] == "N":
                n_count += 1
            else:
                null_count += 1
    
    print(f"\nFinal check of QC field in {joined_fc_name}:")
    print(f"Y count: {y_count}")
    print(f"N count: {n_count}")
    print(f"Null count: {null_count}")
    
    print("===== QC FIELD FIX COMPLETED =====")


===== FIXING QC FIELDS - CORRECTED KEY FORMAT =====
Examining the first 5 QC=Y records in the points layer:
Raw values: StreetID='1002', SectionID='5466'
Corrected key: '1002 - 5466'
Raw values: StreetID='1004', SectionID='1438'
Corrected key: '1004 - 1438'
Raw values: StreetID='1006', SectionID='1287'
Corrected key: '1006 - 1287'
Raw values: StreetID='1014', SectionID='4483'
Corrected key: '1014 - 4483'
Raw values: StreetID='1014', SectionID='5947'
Corrected key: '1014 - 5947'
Found 1879 records with QC=Y in points layer

Sampling keys from joined shapefile:
Joined shapefile key: '1002 - 5466'
Is this key in Y keys? True
Joined shapefile key: '1002 - 5467'
Is this key in Y keys? False
Joined shapefile key: '1004 - 4057'
Is this key in Y keys? False
Joined shapefile key: '1004 - 5279'
Is this key in Y keys? False
Joined shapefile key: '1004 - 1438'
Is this key in Y keys? True

Updating QC values in joined shapefile...
Updated key: 1002 - 5466
Updated key: 1004 - 1438
Updated key: 1006